In [1]:
import pandas as pd
import numpy as np
import yaml

# Getting the Data from Config File

In [2]:
config = open("../configs.yaml", 'r')
dictionary = yaml.safe_load(config)

In [3]:
# Data Loading Fields
GRADES_FILENAME = dictionary["data_path"]["grades_filename"]

# Lab Fields
NUM_LABS = dictionary["labs"]["num_labs"]
MAX_LABS = dictionary["labs"]["max_labs"]

# Project Fields
NUM_PROJECTS = dictionary["projects"]["num_projects"]
MAX_PROJECTS = dictionary["projects"]["max_projects"]

NUM_PROJECT_CHECKPOINTS = dictionary["projects"]["num_checkpoints"]
MAX_PROJECT_CHECKPOINTS = dictionary["projects"]["max_checkpoints"]

# Midterm fields
YES_MIDTERM = dictionary["exams"]["midterm"]["enabled"]
MIDTERM_VERSIONS = dictionary["exams"]["midterm"]["versions"]
MIDTERM_BONUS = dictionary["exams"]["midterm"]["bonus"]

# Final Fields
YES_FINAL = dictionary["exams"]["final"]["enabled"]
FINAL_VERSIONS = dictionary["exams"]["final"]["versions"]

# Discussion Fields
NUM_DI = dictionary["discussions"]["num_dis"]
MAX_DI = dictionary["discussions"]["max_dis"]

# Number of dropped assignments per category (set to 0 for no drops)
# For example, if NUM_DROPS = 2, then 2 Labs and 2 Homeworks will be dropped
# Note: Set this to 0 until the end of the quarter
NUM_DROPS = dictionary["drop_policy"]["num_drops"]

OVERALL_EC = dictionary["extra_credit"]["overall"]

ASSIGNMENT_WEIGHTS = dictionary["assignment_weights"]
DISCUSSION_ASSIGNMENT_WEIGHTS = dictionary["discussion_assignment_weights"]

# Late Submission Grace Period (3 Minutes)
GRACE_PERIOD = 3 / 60 

# Reading in the Current Grade CSV

In [4]:
df = pd.read_csv(GRADES_FILENAME)
discussion_and_lecture_attendence = pd.read_csv('../data/attendance_v2.csv').drop(columns=['Email', 'Name'])
df = df.merge(discussion_and_lecture_attendence, on='SID')

# Create name column if not already present
if 'Name' not in df.columns:
    df['Name'] = df['First Name'] + ' ' + df['Last Name']
    df = df.drop(['First Name', 'Last Name'], axis=1)

# Lab 1 Does not have Redemption Assignement 

In [5]:
# At the beginning of the quarter, we did not do Lab 1 Redemption
def redemption_score(lateness, score):
    if lateness != "00:00:00":
        return score
    else:
        return np.nan

df["Lab 1 Redemption"] = df.apply(
    lambda row: redemption_score(row["Lab 1 - Lateness (H:M:S)"], row["Lab 1"]), axis=1
)
df["Lab 1 Redemption - Max Points"] = df["Lab 1 - Max Points"]

In [6]:
df

,Name,SID,Email,Lab 1,Lab 1 - Max Points,Lab 1 - Submission Time,Lab 1 - Lateness (H:M:S),Project 1 Checkpoint,Project 1 Checkpoint - Max Points,Project 1 Checkpoint - Submission Time,...,Discussion 8 Worksheet - Lateness (H:M:S),Lab 7 Redemption,Lab 7 Redemption - Max Points,Lab 7 Redemption - Submission Time,Lab 7 Redemption - Lateness (H:M:S),Total Lateness (H:M:S),disc_count,lecture_count,Lab 1 Redemption,Lab 1 Redemption - Max Points
0,Abdulrahim Ham,A17531851,amham@ucsd.edu,129.0,133.0,2024-04-10 13:16:54 -0700,00:00:00,13.0,13.0,2024-04-12 17:15:27 -0700,...,00:00:00,NaN,111.0,NaN,00:00:00,00:00:00,1.0,6.0,NaN,133.0
1,Aditya Surapaneni,A18134364,assurapaneni@ucsd.edu,127.0,133.0,2024-04-10 18:40:28 -0700,00:00:00,13.0,13.0,2024-04-11 19:37:51 -0700,...,00:00:00,NaN,111.0,NaN,00:00:00,00:00:00,6.0,7.0,NaN,133.0
2,Adrian Apsay,A17384335,adapsay@ucsd.edu,129.0,133.0,2024-04-10 23:04:53 -0700,00:00:00,13.0,13.0,2024-04-12 23:57:07 -0700,...,00:00:00,NaN,111.0,NaN,00:00:00,00:00:00,7.0,3.0,NaN,133.0
3,Ahmad Ageel,A17500501,aageel@ucsd.edu,NaN,133.0,NaN,00:00:00,NaN,13.0,NaN,...,00:00:00,NaN,111.0,NaN,00:00:00,00:00:00,0.0,0.0,NaN,133.0
4,Albert Henderson,A16384282,adhender@ucsd.edu,126.0,133.0,2024-04-11 11:33:48 -0700,00:00:00,12.0,13.0,2024-04-12 23:26:02 -0700,...,00:00:00,NaN,111.0,NaN,00:00:00,00:00:00,0.0,0.0,NaN,133.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,Zachary Thomason,A18090378,zthomason@ucsd.edu,123.0,133.0,2024-04-12 23:01:49 -0700,47:02:49,13.0,13.0,2024-04-12 22:57:28 -0700,...,00:00:00,NaN,111.0,NaN,00:00:00,47:02:49,4.0,2.0,123.0,133.0
145,Zijin Qin,A17358491,z2qin@ucsd.edu,110.0,133.0,2024-04-10 23:32:06 -0700,00:00:00,13.0,13.0,2024-04-12 13:59:58 -0700,...,00:00:00,NaN,111.0,NaN,00:00:00,47:35:09,6.0,5.0,NaN,133.0
146,Zixun Zhang,A16985661,ziz046@ucsd.edu,128.0,133.0,2024-04-10 18:19:41 -0700,00:00:00,13.0,13.0,2024-04-12 20:09:39 -0700,...,00:00:00,NaN,111.0,NaN,00:00:00,00:00:00,0.0,0.0,NaN,133.0
147,Ziyao Zhou,A17268660,ziz075@ucsd.edu,133.0,133.0,2024-05-08 19:39:03 -0700,667:40:03,13.0,13.0,2024-04-12 23:40:46 -0700,...,00:00:00,NaN,111.0,NaN,00:00:00,667:40:03,6.0,4.0,133.0,133.0


# Calculating the Grade for each Assignment

In [7]:
def get_proportion_assignment(r, assignment_name):
    num = r[assignment_name]
    den = r[assignment_name + ' - Max Points']
    
    return num / den

In [8]:
for i in range(1, NUM_LABS + 1):
    assignment_name = f"Lab {i}"
    df[assignment_name + " Grade"] = df.apply(
        get_proportion_assignment, axis=1, assignment_name=assignment_name
    )
    if i == 1:
        pass

    assignment_name = f"Lab {i} Redemption"
    df[assignment_name + " Grade"] = df.apply(
        get_proportion_assignment, axis=1, assignment_name=assignment_name
    )

In [9]:
for i in range(1, NUM_PROJECTS + 1):
    assignment_name = f"Project {i}"
    df[assignment_name + " Grade"] = df.apply(
        get_proportion_assignment, axis=1, assignment_name=assignment_name
    )

    assignment_name = f"Project {i} Redemption"
    df[assignment_name + " Grade"] = df.apply(
        get_proportion_assignment, axis=1, assignment_name=assignment_name
    )

In [10]:
for i in range(1, NUM_PROJECT_CHECKPOINTS+1):
    assignment_name = f'Project {i} Checkpoint'
    df[assignment_name + ' Final Grade'] = df.apply(get_proportion_assignment, axis=1, assignment_name=assignment_name)

# Calculating Final Grade with Redemption Assignments

In [11]:
def get_final_score(assignment_name):
    df[f"{assignment_name} Final Grade"] = df[f"{assignment_name} Grade"] + (
        (df[f"{assignment_name} Redemption Grade"] - df[f"{assignment_name} Grade"]).apply(
            lambda x: x if x > 0 else 0
        )
        * 0.8
    ).fillna(0)

In [12]:
redemptions_assignments = (
    [f"Project {i+1}" for i in range(0, NUM_PROJECTS)]
    + [f"Lab {i+1}" for i in range(0, NUM_LABS)]
)
for assignment in redemptions_assignments:get_final_score(assignment) 

# Finding Dropped Lab

In [13]:
def find_dropped_labs(df):
    lab_scores = df[[f'Lab {i} Final Grade' for i in range(1, NUM_LABS+1)]].fillna(0)
    dropped_labs = np.argsort(np.array(lab_scores))[:NUM_DROPS]
    dropped_labs = np.sort(dropped_labs) + 1
    dropped_labs = [f'Lab {l}' for l in dropped_labs]
    return ', '.join(dropped_labs)

df['Dropped Labs'] = df.apply(find_dropped_labs, axis=1)

# Calculating Midterm and Final Grades

In [14]:
if YES_MIDTERM:
    df['Midterm Exam Grade Pre-EC'] = df['Midterm'] / df['Midterm - Max Points']
    df['Midterm Mean'] = df['Midterm Exam Grade Pre-EC'].dropna().mean()
    df['Midterm std'] = df['Midterm Exam Grade Pre-EC'].dropna().std(ddof=0)
    df['Midterm Z-Score'] = (df['Midterm Exam Grade Pre-EC'] - df['Midterm Mean']) / df['Midterm std']
    df['Midterm Exam Grade Post-EC'] = df['Midterm Exam Grade Pre-EC'] + MIDTERM_BONUS  
    if YES_FINAL:
        ...
    else:
       df['Midterm Exam Grade Post-Redemption Post-EC'] = df['Midterm Exam Grade Post-EC'] 

if YES_FINAL:
    # Have to wrok on Midterm Redemption 
    df['Final Exam Grade'] = df['Final'] / df['Final - Max Points']

# Finding Students who are eligible for Discussion Section

In [15]:
df['Elgible for Discussion'] = (df['disc_count'] >= 8) & (df['lecture_count'] >= 8)

# Calculating the Maximum Possible Grade Right Now 

In [16]:
df["Max Possible Points"] = (
    NUM_LABS / MAX_LABS * ASSIGNMENT_WEIGHTS["lab"]
    + NUM_PROJECTS / MAX_PROJECTS * ASSIGNMENT_WEIGHTS["project"]
    + NUM_PROJECT_CHECKPOINTS / MAX_PROJECT_CHECKPOINTS * ASSIGNMENT_WEIGHTS["project_checkpoint"]
)
if YES_MIDTERM:
    df["Max Possible Points"] += ASSIGNMENT_WEIGHTS["midterm_exam"]
if YES_FINAL:
    df["Max Possible Points"] += ASSIGNMENT_WEIGHTS["final_exam"]
df["Max Possible Points"] *= 100
df['Current Max Possible Score'] = df["Max Possible Points"].max()

# Calculating Scores for Each Assignment Group

## 1 Lab is Dropped

In [17]:
def calculate_lab_mean(row):
    exclude_col = row['Dropped Labs']
    lab_columns = [f"Lab {i+1} Final Grade" for i in range(0, NUM_LABS)]
    columns_to_include = [col for col in lab_columns if col not in [exclude_col, 'Exclude']]
    row_mean = row[columns_to_include].mean()
    return row_mean

In [18]:
df['Lab Average'] = df[[f"Lab {i+1} Final Grade" for i in range(0, NUM_LABS)]].fillna(0).mean(axis=1)
df['Project Average'] = df[[f"Project {i+1} Final Grade" for i in range(0, NUM_PROJECTS)]].fillna(0).mean(axis=1)
df['Project Checkpoint Average'] = df[[f"Project {i+1} Checkpoint Final Grade" for i in range(0, NUM_PROJECT_CHECKPOINTS)]].fillna(0).mean(axis=1)
if YES_MIDTERM:
    df['Midterm Average'] = df['Midterm Exam Grade Post-Redemption Post-EC']
if YES_FINAL:
    df['Final Average'] = df['Final Final']

# Claculating Scores for Each Student Without Discussion

In [19]:
df['Overall Score without Discussion'] = (
    df['Lab Average'] * NUM_LABS / MAX_LABS * ASSIGNMENT_WEIGHTS["lab"]
    + df['Project Average'] * NUM_PROJECTS / MAX_PROJECTS * ASSIGNMENT_WEIGHTS["project"]
    + df['Project Checkpoint Average'] * NUM_PROJECT_CHECKPOINTS / MAX_PROJECT_CHECKPOINTS * ASSIGNMENT_WEIGHTS["project_checkpoint"]
)
if YES_MIDTERM:
    df['Overall Score without Discussion'] += df['Midterm Average'] * ASSIGNMENT_WEIGHTS["midterm_exam"]
if YES_FINAL:
    df['Overall Score without Discussion'] += df['Final Average'] * ASSIGNMENT_WEIGHTS["final_exam"]

df['Overall Score without Discussion'] *= 100

df['Overall Score without Discussion'].max()

53.12932079596517

# Claculating Scores for Each Student With Discussion

In [20]:
df['Overall Score with Discussion'] = (
    df['Lab Average'] * NUM_LABS / MAX_LABS * DISCUSSION_ASSIGNMENT_WEIGHTS["lab"]
    + df['Project Average'] * NUM_PROJECTS / MAX_PROJECTS * DISCUSSION_ASSIGNMENT_WEIGHTS["project"]
    + df['Project Checkpoint Average'] * NUM_PROJECT_CHECKPOINTS / MAX_PROJECT_CHECKPOINTS * DISCUSSION_ASSIGNMENT_WEIGHTS["project_checkpoint"]
)
if YES_MIDTERM:
    df['Overall Score with Discussion'] += df['Midterm Average'] * DISCUSSION_ASSIGNMENT_WEIGHTS["midterm_exam"]
if YES_FINAL:
    df['Overall Score with Discussion'] += df['Final Average'] * DISCUSSION_ASSIGNMENT_WEIGHTS["final_exam"]

df['Overall Score with Discussion'] + DISCUSSION_ASSIGNMENT_WEIGHTS["discussion"]

df['Overall Score with Discussion'] *= 100
df['Overall Score with Discussion'].max()

51.899320795965174

# Finding Current Overall Score

In [21]:
df["Overall Score"] = df.apply(
    lambda row: max(
        row["Overall Score with Discussion"],
        row["Overall Score without Discussion"] * row["Elgible for Discussion"],
    ),
    axis=1,
)

# Calculating Current Grade(Percent / Total Point Possible) for Each Student

In [22]:
df['Current Score'] = df['Overall Score'] / df['Max Possible Points'] * 100


In [23]:
df['Current Score']

0      70.760643
1      76.907617
2      73.163005
3      32.081660
4      65.600425
         ...    
144    73.802447
145    65.554342
146    81.004251
147    84.295876
148    69.273980
Name: Current Score, Length: 149, dtype: float64

In [24]:
df.to_csv('../data/grades_for_grade_report.csv', index=False)